In [2]:
import numpy as np
import pandas as pd
import zipfile
from tabulate import tabulate
from typing import Optional
from functions_dependency_test import heatmap_chi, heatmap_mutual_info
from Data_transformations import feature_cleaning, values_cleaning
from preprocess import clean_dataframe

In [3]:
raw_data = pd.read_csv('Murder_Data.zip', index_col=0, compression='zip')

df = clean_dataframe(raw_data)
df = df.drop(['Solved'], axis=1)

/Users/thomas/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
p_of_chi = heatmap_chi(df, return_df=True)

In [7]:
mi_scores = heatmap_mutual_info(df.astype(str), return_df=True)

In [11]:
from sklearn.metrics import normalized_mutual_info_score
import plotly.figure_factory as ff
sample_inputs = ['State', 'Weapon', 'VicCount', 'VicRace', 'VicSex', 'Circumstance']
sample_outputs = ['OffSex', 'OffRace', 'OffCount', 'Relationship']
mutual_info_scores = np.zeros(shape=(6, 4))
for i, input in enumerate(sample_inputs):
    for j, output in enumerate(sample_outputs):
        mutual_info = normalized_mutual_info_score(df[input], df[output])
        mutual_info_scores[i][j] = mutual_info

mi_rounded = np.around(mutual_info_scores, decimals=4)
fig = ff.create_annotated_heatmap(mutual_info_scores,
                x=sample_outputs,
                y=sample_inputs,
                annotation_text=mi_rounded
            )
#fig.update_xaxes(side="top")
fig.update_layout( title_text='Mutual Information scores',
                    autosize=False,
                    width=1000,
                    height=1000
                    )
fig.show()

We at State/OffSex
We at State/OffRace
We at State/OffCount
We at State/Relationship
We at Weapon/OffSex
We at Weapon/OffRace
We at Weapon/OffCount
We at Weapon/Relationship
We at VicCount/OffSex
We at VicCount/OffRace
We at VicCount/OffCount
We at VicCount/Relationship
We at VicRace/OffSex
We at VicRace/OffRace
We at VicRace/OffCount
We at VicRace/Relationship
We at VicSex/OffSex
We at VicSex/OffRace
We at VicSex/OffCount
We at VicSex/Relationship
We at Circumstance/OffSex
We at Circumstance/OffRace
We at Circumstance/OffCount
We at Circumstance/Relationship
